In [ ]:
!pip install qiskit qutip numpy matplotlib torch tqdm scikit-learn einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.8/31.8 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
from qutip import basis, ket2dm, Qobj

zero = basis(2, 0)  # |0> = [1, 0]
one = basis(2, 1)   # |1> = [0, 1]

# Example: create a pure superposition state
psi = (zero + one).unit()  # |ψ> = (|0> + |1>)/√2
pure_dm = ket2dm(psi)      # Density matrix ρ = |ψ><ψ|

# Create a mixed state as a weighted sum of pure states
p = 0.7213214678631  # probability of being |ψ>
rho_mixed = p * pure_dm + (1 - p) * ket2dm(one)

print("Mixed-state density matrix:")
print(rho_mixed)

Mixed-state density matrix:
Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.36066073 0.36066073]
 [0.36066073 0.63933927]]


In [ ]:
from qutip import basis, ket2dm
import numpy as np

def pure_superposition_states(n_samples=10000):
    pure_states = []
    for _ in range(n_samples):
        # Random angles on Bloch sphere
        theta = np.arccos(2*np.random.rand() - 1)  # polar angle [0, pi]
        phi = 2*np.pi*np.random.rand()             # azimuthal angle [0, 2pi]

        # Construct random pure superposition state:
        # |ψ> = cos(theta/2)|0> + exp(i*phi)*sin(theta/2)|1>
        psi = np.cos(theta/2)*basis(2,0) + np.exp(1j*phi)*np.sin(theta/2)*basis(2,1)

        # Convert to density matrix representation (optional)
        rho_pure = ket2dm(psi)
        pure_states.append(rho_pure)

    return pure_states


In [ ]:
pure_state_data = pure_superposition_states()
print(pure_state_data[0])

Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.12521085+0.j         0.0594765 +0.32556971j]
 [0.0594765 -0.32556971j 0.87478915+0.j        ]]


In [ ]:
from qutip import Qobj, ket2dm, tensor, ptrace
import numpy as np # Import numpy

def generate_clones(pure_state_data):
  cloned_pairs = []
  U_clone = Qobj([[1,0,0,0],
                [0,1/np.sqrt(2),1/np.sqrt(2),0],
                [0,1/np.sqrt(2),-1/np.sqrt(2),0],
                [0,0,0,1]], dims=[[2, 2], [2, 2]]) # Corrected dimensions
  ancilla = ket2dm(basis(2,0))          # Blank qubit |0>
  for rho in pure_state_data:  # mixed_states is your list of 10k arrays
    rho_joint = tensor(rho, ancilla)      # Combine system

    rho_cloned = U_clone * rho_joint * U_clone.dag()  # apply unitary

    clone1 = ptrace(rho_cloned, 0)
    clone2 = ptrace(rho_cloned, 1)
    cloned_pairs.append((clone1, clone2))
  return cloned_pairs

In [ ]:
generated_clones = generate_clones(pure_state_data)
print(generated_clones[0])

(Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.56260542+0.j         -0.04205624-0.23021255j]
 [-0.04205624+0.23021255j  0.43739458+0.j        ]], Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.56260542+0.j         0.04205624+0.23021255j]
 [0.04205624-0.23021255j 0.43739458+0.j        ]])


In [ ]:
import numpy as np
from qutip import sigmax, sigmay, sigmaz, Qobj

def qobj_to_bloch(rho):
    """Convert a single QuTiP Qobj density matrix to a Bloch vector."""
    x = np.real((rho * sigmax()).tr())
    y = np.real((rho * sigmay()).tr())
    z = np.real((rho * sigmaz()).tr())
    return np.array([x, y, z])

def convert_pairs_to_bloch(pair_list):
    """
    Convert a list of (rho1, rho2) pairs to Bloch vectors.

    Returns two arrays:
      - bloch1: first clones
      - bloch2: second clones
    """
    bloch1 = []
    bloch2 = []

    for rho1, rho2 in pair_list:
        bloch1.append(qobj_to_bloch(rho1))
        bloch2.append(qobj_to_bloch(rho2))

    return np.array(bloch1), np.array(bloch2)

# Example usage
# pair_list = [(rho1_clone1, rho1_clone2), (rho2_clone1, rho2_clone2), ...]
bloch_clone1, bloch_clone2 = convert_pairs_to_bloch(generated_clones)

print(bloch_clone1.shape)  # (10000, 3)
print(bloch_clone2.shape)  # (10000, 3)


(10000, 3)
(10000, 3)


In [ ]:
def convert_original_to_bloch(original_list):
    bloch = []

    for rho in original_list:
        bloch.append(qobj_to_bloch(rho))


    return np.array(bloch)

bloch_original = convert_original_to_bloch(pure_state_data)

print(bloch_original.shape)  # (10000, 3)

(10000, 3)


In [ ]:
from qutip import fidelity

def average_fidelity(y_pred_np, y_true_np):
    """
    Compute approximate average fidelity between predicted and true states.
    Both are Bloch vectors (not density matrices).
    """
    # Convert Bloch vectors back to density matrices:
    fids = []
    for r_pred, r_true in zip(y_pred_np, y_true_np):
        rho_pred = 0.5 * (Qobj(np.eye(2)) + r_pred[0]*sigmax() + r_pred[1]*sigmay() + r_pred[2]*sigmaz())
        rho_true = 0.5 * (Qobj(np.eye(2)) + r_true[0]*sigmax() + r_true[1]*sigmay() + r_true[2]*sigmaz())
        fids.append(fidelity(rho_pred, rho_true))
    return np.mean(fids)


In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split


x_train_orig, x_test_orig, x_train_c1, x_test_c1, x_train_c2, x_test_c2 = train_test_split(
    bloch_original, bloch_clone1, bloch_clone2, test_size=0.2, random_state=42
)

# Stack clones to create inputs and targets
X_train = np.vstack([x_train_c1, x_train_c2]).astype(np.float32)
y_train = np.vstack([x_train_orig, x_train_orig]).astype(np.float32)

X_test = np.vstack([x_test_c1, x_test_c2]).astype(np.float32)
y_test = np.vstack([x_test_orig, x_test_orig]).astype(np.float32)


# -------------------------------
# PyTorch Dataset
# -------------------------------
class BlochDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create datasets
train_dataset = BlochDataset(X_train, y_train)
test_dataset = BlochDataset(X_test, y_test)

# DataLoaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
import torch.nn as nn

class BlochDenoiser(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(3, 16),
            nn.ReLU(),
            nn.Linear(16, 16),
            nn.ReLU(),
            nn.Linear(16, 3)
        )

    def forward(self, x):
        return self.net(x)

model = BlochDenoiser()
criterion = lambda x, y: 1 - torch.nn.functional.cosine_similarity(x, y).mean()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

n_epochs = 50

for epoch in range(n_epochs):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * X_batch.size(0)
    avg_loss = total_loss / len(train_loader.dataset)

    # Evaluate on test set
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            output = model(X_batch)
            test_loss += criterion(output, y_batch).item() * X_batch.size(0)
    avg_test_loss = test_loss / len(test_loader.dataset)

    print(f"Epoch {epoch+1}/{n_epochs} | Train Loss: {avg_loss:.6f} | Test Loss: {avg_test_loss:.6f}")

Epoch 1/50 | Train Loss: 0.648037 | Test Loss: 0.521308
Epoch 2/50 | Train Loss: 0.517868 | Test Loss: 0.515644
Epoch 3/50 | Train Loss: 0.511646 | Test Loss: 0.515906
Epoch 4/50 | Train Loss: 0.507924 | Test Loss: 0.516829
Epoch 5/50 | Train Loss: 0.505536 | Test Loss: 0.517339
Epoch 6/50 | Train Loss: 0.503564 | Test Loss: 0.507407
Epoch 7/50 | Train Loss: 0.502172 | Test Loss: 0.510312
Epoch 8/50 | Train Loss: 0.501118 | Test Loss: 0.511021
Epoch 9/50 | Train Loss: 0.502173 | Test Loss: 0.505827
Epoch 10/50 | Train Loss: 0.500609 | Test Loss: 0.507052
Epoch 11/50 | Train Loss: 0.500082 | Test Loss: 0.507981
Epoch 12/50 | Train Loss: 0.500501 | Test Loss: 0.506956
Epoch 13/50 | Train Loss: 0.499861 | Test Loss: 0.508306
Epoch 14/50 | Train Loss: 0.501382 | Test Loss: 0.510288
Epoch 15/50 | Train Loss: 0.498950 | Test Loss: 0.507017
Epoch 16/50 | Train Loss: 0.499973 | Test Loss: 0.509069
Epoch 17/50 | Train Loss: 0.499236 | Test Loss: 0.507692
Epoch 18/50 | Train Loss: 0.497675 | Tes

In [ ]:
import torch
import numpy as np

# Make sure the model is in evaluation mode
model.eval()

# Convert your test data to torch tensors if not already
X_test_tensor = torch.from_numpy(X_test)  # shape: (n_samples*2, 3)
y_test_tensor = torch.from_numpy(y_test)  # shape: (n_samples*2, 3)

# Run the model on test data
with torch.no_grad():
    y_pred = model(X_test_tensor)

# Convert predictions back to NumPy
y_pred_np = y_pred.numpy()
y_pred_np /= np.maximum(np.linalg.norm(y_pred_np, axis=1, keepdims=True), 1e-8)


avg_fid = average_fidelity(y_pred_np, y_test)
print(f"Average fidelity on test set: {avg_fid:.4f}")


Average fidelity on test set: 0.8558


In [ ]:
mse = np.mean((y_pred_np - y_test)**2)
print(f"Test MSE: {mse:.8f}")


Test MSE: 0.33600813


In [ ]:
avg_fid = average_fidelity(y_pred_np, y_test)
print(f"Average fidelity on test set: {avg_fid:.4f}")


Average fidelity on test set: 0.8558


In [ ]:
import numpy as np

def bloch_fidelity_batch(rho_array, sigma_array):
    """
    Compute fidelities between two arrays of Bloch vectors.

    Parameters:
        rho_array: np.array of shape (n_samples, 3) - true vectors
        sigma_array: np.array of shape (n_samples, 3) - predicted vectors

    Returns:
        fidelities: np.array of shape (n_samples,)
    """
    r_dot = np.sum(rho_array * sigma_array, axis=1)           # dot product for each sample
    r_norm = np.linalg.norm(rho_array, axis=1)               # magnitude of true vectors
    s_norm = np.linalg.norm(sigma_array, axis=1)             # magnitude of predicted vectors
    fidelities = 0.5 * (1 + r_dot + np.sqrt(1 - r_norm**2) * np.sqrt(1 - s_norm**2))
    return fidelities

# Example usage:

fidelities = bloch_fidelity_batch(y_test, y_pred_np)

# Average fidelity over all test samples
print(f"Average fidelity: {np.mean(fidelities):.6f}")

# Optional: inspect first few
for i in range(5):
    print(f"True: {y_test[i]}, Predicted: {y_pred_np[i]}, Fidelity: {fidelities[i]:.6f}")


Average fidelity: nan
True: [-0.8157814  -0.57257915 -0.08157089], Predicted: [-0.15255636 -0.2134053  -0.9649791 ], Fidelity: 0.662679
True: [-0.87747955 -0.24022867  0.41511422], Predicted: [-0.08460446  0.0230076   0.99614894], Fidelity: 0.741114
True: [-0.7132756   0.39823484  0.57675546], Predicted: [-0.06165397 -0.01686644  0.9979551 ], Fidelity: 0.806418
True: [ 0.4395178   0.40728974 -0.80058676], Predicted: [-0.0270256  -0.02264734 -0.99937814], Fidelity: 0.889493
True: [ 0.5645015  -0.81346977 -0.14001752], Predicted: [-0.2930257  -0.10119441 -0.9507342 ], Fidelity: 0.525012


/tmp/ipython-input-1052572072.py:17: RuntimeWarning: invalid value encountered in sqrt
  fidelities = 0.5 * (1 + r_dot + np.sqrt(1 - r_norm**2) * np.sqrt(1 - s_norm**2))
